**Chapter 10. 생성 네트워크**


# GAN 실습
1. 데이터 읽어서 형태 변형
2. 생성모델 구성
3. 판별모델 구성
4. 생성-판별 연결 모델 구성
5. 학습

# 1. 데이터 읽기, Reshape와 정규화

In [ ]:
import pandas as pd
import numpy as np
import tensorflow.keras as keras
from keras import Sequential, layers, optimizers
import matplotlib.pyplot as plt

In [ ]:
d = pd.read_csv('/content/sample_data/mnist_test.csv')
d = np.array(d)
y_train = d[:,1:].reshape(-1, 28, 28)
y_train = y_train/255

# 2. 네트워크의 구성

In [ ]:
# 1) 생성 네트워크의 구성
model_gen = Sequential()
model_gen.add(layers.Dense(units=3136, activation='relu'))
model_gen.add(layers.BatchNormalization())
model_gen.add(layers.Reshape((7,7,64)))
model_gen.add(layers.UpSampling2D((2,2)))
model_gen.add(layers.Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
model_gen.add(layers.BatchNormalization())
model_gen.add(layers.UpSampling2D((2,2)))
model_gen.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
model_gen.add(layers.BatchNormalization())
model_gen.add(layers.Conv2D(filters=1, kernel_size=3, padding='same', activation='sigmoid'))
# model_gen.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=0.002), metrics=['accuracy'])
model_gen.compile(loss='min_squrraed_error', optimizer=optimizers.RMSprop(learning_rate=0.002), metrics=['accuracy'])

In [ ]:
# 2) 판별 모델의 구성
model_disc = Sequential()
model_disc.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', input_shape=(28,28,1), activation='relu'))
model_disc.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
model_disc.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model_disc.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
model_disc.add(layers.Conv2D(filters=16, kernel_size=3, padding='same', activation='relu'))
model_disc.add(layers.MaxPooling2D(pool_size=(3,3), strides=2))
model_disc.add(layers.Flatten())
model_disc.add(layers.Dense(units=1, activation='sigmoid'))
model_disc.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [ ]:
# 3) 생성 모델과 판별 모델의 연결
model_comb = Sequential()
model_comb.add(model_gen)
model_comb.add(model_disc)
model_disc.trainable = False
model_comb.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(learning_rate=0.001), metrics=['accuracy'])

# 3. 학습

In [ ]:
# 배치 크기 등 설정
nOrig = 64
nGen = nOrig
vector_size = 10
nEpoch = 2000

# 반복
for i in range(nEpoch):
  # 이미지 생성
  y_gen = np.zeros((nGen,28,28))
  test_input = np.random.rand(nGen,vector_size)
  for j in range(nGen):
    o = model_gen.predict(test_input[j,:].reshape(1,10))
    o = o.reshape((28,28))
    y_gen[j,:] = o
  y_gen = np.expand_dims(y_gen, -1)

  # 원본 이미지 선택
  idx = np.array(range(y_train.shape[0]))
  np.random.shuffle(idx)
  idx = idx[:nOrig]
  y_orig = y_train[idx,:,:]
  y_orig = np.expand_dims(y_orig, -1)

  # 원본 이미지 - 생성 이미지 결합
  test_img = np.concatenate((y_gen, y_orig), 0)
  test_target = np.concatenate((np.zeros(y_gen.shape[0]), np.ones(y_orig.shape[0])), 0)

  # 판별자 학습
  loss_disc = model_disc.train_on_batch(test_img, test_target)

  # 생성자 학습
  loss_gen = model_comb.train_on_batch(test_input, np.ones(test_input.shape[0]))

In [ ]:
# 배치 크기 등 설정
nOrig = 64
nGen = nOrig
vector_size = 10
nEpoch = 2000

# 반복
for i in range(nEpoch):
  # 이미지 생성
  y_gen = np.zeros((nGen,28,28))
  test_input = np.random.rand(nGen,vector_size)
  for j in range(nGen):
    o = model_gen.predict(test_input[j,:].reshape(1,10))
    o = o.reshape((28,28))
    y_gen[j,:] = o
  y_gen = np.expand_dims(y_gen, -1)

  # 원본 이미지 선택
  idx = np.array(range(y_train.shape[0]))
  np.random.shuffle(idx)
  idx = idx[:nOrig]
  y_orig = y_train[idx,:,:]
  y_orig = np.expand_dims(y_orig, -1)

  # 원본 이미지 - 생성 이미지 결합
  test_img = np.concatenate((y_gen, y_orig), 0)
  test_target = np.concatenate((np.zeros(y_gen.shape[0]), np.ones(y_gen.shape[0])), 0)

  # 판별자 학습
  loss_disc = model_disc.train_on_batch(test_img, test_target)

  # 생성자 학습
  loss_gen = model_comb.train_on_batch(test_input, np.ones(test_input.shape[0]))

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 32ms/step


# 4. 이미지 생성과 생성된 이미지의 확인

In [ ]:
from matplotlib import figure
flg = plt.figure(figsize=[20,10])
test_input = np.random.rand(nGen, vector_size)
for i in range(nGen):
  o = model_gen.predict(test_input[i,:].reshape(1,vector_size))
  o = o.reshape((28,28))
  ax = flg.add_subplot(8,8,i+1)
  ax.imshow(o)